<a href="https://colab.research.google.com/github/simonefmr/mvp_pucrio/blob/main/Sprint2_MVP_B_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Pós Graduação Ciência de Dados - PUC Rio**
## **MVP Sprint II B - Deep Learning**

##Aluna *Simone de Fátima Marques Ramos*

## ***AAA***

About Dataset
O Dataset
O dataset imdb é um dos mais famosos benchmarks para modelos de classificação de textos que existe. Dificilmente não será citado em papers que proponham trabalhos envolvendo este tipo de tarefa.

A tradução
Recentemente eu tive a ideia de traduzir o conteúdo do dataset para o português, para atender a uma necessidade específica. Como vi que o resultado ficou a contento, achei que poderia ser útil para mais alguém. Para que você possa testá-lo, tente adaptar o código disponibilizado neste post: https://medium.com/luisfredgs/an%C3%A1lise-de-sentimentos-com-redes-neurais-recorrentes-lstm-a5352b21e6aa

https://www.kaggle.com/datasets/luisfredgs/imdb-ptbr?resource=download

Você deverá treinar um modelo baseado em Deep Learning para resolver um problema de
visão computacional ou processamento de linguagem natural, iniciando na carga e
preparação dos dados, incluindo a separação entre treino e teste, modelagem, otimização
de hiperparâmetros, até a avaliação e comparação de resultados dos modelos treinados.
b.

**Definição do Problema**

Objetivo: entender e descrever claramente o problema que está sendo resolvido.

• Qual é a descrição do problema?
• Você tem premissas ou hipóteses sobre o problema? Quais?
• Que restrições ou condições foram impostas para selecionar os dados?
• Descreva o seu dataset (atributos, imagens, anotações, etc).

**Preparação de Dados**

Objetivo: realizar operações de preparação dos dados.

• Separe o dataset entre treino e teste (e validação, se aplicável).
• Faz sentido utilizar um método de validação cruzada? Justifique se não utilizar.
Verifique quais operações de transformação de dados (como normalização e padronização,
transformação de imagens em tensores) são mais apropriadas para o seu problema e salve
visões diferentes do seu dataset para posterior avaliação dos modelos.
•
Refine a quantidade de atributos disponíveis, realizando o processo de feature selection de forma adequada.

**Modelagem e treinamento:**

Objetivo: construir modelos para resolver o problema em questão.

Selecione os algoritmos mais indicados para o problema e dataset escolhidos, justificando as
suas escolhas.

• Há algum ajuste inicial para os hiperparâmetros?
• O modelo foi devidamente treinado? Foi observado problema de underfitting?
É possível otimizar os hiperparâmetros de algum dos modelos? Se sim, faça-o, justificando
todas as escolhas.
• Há algum método avançado ou mais complexo que possa ser avaliado?
• Posso criar um comitê de modelos diferentes para o problema (ensembles)?

**Avaliação de Resultados:**

Objetivo: analisar o desempenho dos modelos gerados em dados não vistos (com a base de teste)

• Selecione as métricas de avaliação condizentes com o problema, justificando.
• Treine o modelo escolhido com toda a base de treino, e teste-o com a base de teste.
• Os resultados fazem sentido?
• Foi observado algum problema de overfitting?
• Compare os resultados de diferentes modelos.
• Descreva a melhor solução encontrada, justificando.

In [2]:
# Importando bibliotecas

import pandas as pd

In [25]:
# Importando Dataset

url = ('https://raw.githubusercontent.com/simonefmr/mvp_pucrio/main/review_treino_teste.csv')
data = pd.read_csv(url, sep=';', error_bad_lines=False)
data.head()

<ipython-input-25-a991222768ae>:4: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv(url, sep=';', error_bad_lines=False)


,sentiment,review
0,Negative,I had no background knowledge of this movie be...
1,Negative,I am a huge Jane Austen fan and I ordered the ...
2,Negative,Nothing to say but Wow! Has anyone actually ha...
3,Negative,i like Jane Austin novels. I love Pride and Pr...
4,Negative,In this day and age of incredible special movi...


In [26]:
data.shape

(9800, 2)